In [ ]:
from google.colab import drive   # colab 사용시 mount 하기 위한 용도. local 에서는 불필요

drive.mount('/content/gdrive/')  # colab 사용시 mount 하기 위한 용도. local 에서는 불필요

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
%tensorflow_version 1.x

import tensorflow as tf
import os

working_dir = 'tensorflow_2.x_working_dir'

colab_default_dir = '/content/gdrive/My Drive/Colab Notebooks/'

original_dir = os.getcwd()

try:

  os.chdir(colab_default_dir)

  if not os.path.exists(working_dir):
    os.mkdir(working_dir)

  os.chdir(working_dir)
  print("current dir = ", os.getcwd())

except Exception as err:
  
  os.chdir(original_dir)
  print(str(err))

current dir =  /content/gdrive/My Drive/Colab Notebooks/tensorflow_2.x_working_dir


In [ ]:
class DataGeneration:
    
    # target_position = 0 (첫번째열이 정답데이터), target_position=-1 (마지막열이 정답데이터)
    def __init__(self, name, file_path, seperation_rate, target_position=-1):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        if (target_position == -1  or  target_position == 0):      
            self.target_position = target_position
        
        else:
            err_str = 'target_position must be -1 or 0'            
            raise Exception(err_str)    
            
    
    # print data target distribution 
    # str_of_kind : 'original data' or  'training data'  or  'test data'
    def __display_target_distribution(self, data, str_of_kind='original data'):
        
        print('=======================================================================================================')
        
        target_data = data[ :, self.target_position ]
        
        # numpy.unique() 사용하여 loaded data target 분포 확인
        unique, counts = np.unique(target_data, return_counts=True)

        unique_target = []
    
        for index in range(len(unique)):
        
            print('[DataGeneration] unique number of ' + str_of_kind + ' = ', unique[index], ', count = ', counts[index])
        
            unique_target.append(unique[index])

        for index in range(len(unique_target)):
        
            print('[DataGeneration] unique number of ' + str_of_kind + ' = ', unique_target[index], ', ratio = ', np.round(100 * counts[index] / (target_data.shape[0]), 2), ' %')
    
        print('=======================================================================================================')
        
        
    # numpy.random.shuffle()  이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기, 파일이 없는 경우 exception 발생

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))

        print("[DataGeneration]  loaded_data.shape = ", loaded_data.shape)
            
        # print the target distribution of original data 
        
        self.__display_target_distribution(loaded_data, 'original data')
        
        
        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        # numpy.random.shuffle 을 이용하여 랜덤하게 데이터 섞기
        np.random.shuffle(loaded_data)
        
        # test_data 는 0 : test_data_num
        
        
        test_data = loaded_data[ 0:test_data_num ]

        # training_data 는 test_data_num 부터 끝까지 
        training_data = loaded_data[ test_data_num: ]

        # display target distribution of generated data 
        
        self.__display_target_distribution(training_data, 'training data')
        
        self.__display_target_distribution(test_data, 'test data')
        
        return training_data, test_data

In [ ]:
# DataGeneration class 이용하여 training data , validation data 생성
seperation_rate = 0.3  # training data 10 % 비율로 validation data 생성
target_position = -1    # 정답은 첫번째 열

try:
    data_obj = DataGeneration('Diabetes', './diabetes.csv', seperation_rate, target_position)

    (training_data, validation_data) = data_obj.generate()
    
    print("training_data.shape = ", training_data.shape)
    print("validation_data.shape = ", validation_data.shape)

except Exception as err:
    print('Exception Occur !!')
    print(str(err))

[DataGeneration]  loaded_data.shape =  (759, 9)
[DataGeneration] unique number of original data =  0.0 , count =  263
[DataGeneration] unique number of original data =  1.0 , count =  496
[DataGeneration] unique number of original data =  0.0 , ratio =  34.65  %
[DataGeneration] unique number of original data =  1.0 , ratio =  65.35  %
[DataGeneration] unique number of training data =  0.0 , count =  192
[DataGeneration] unique number of training data =  1.0 , count =  340
[DataGeneration] unique number of training data =  0.0 , ratio =  36.09  %
[DataGeneration] unique number of training data =  1.0 , ratio =  63.91  %
[DataGeneration] unique number of test data =  0.0 , count =  71
[DataGeneration] unique number of test data =  1.0 , count =  156
[DataGeneration] unique number of test data =  0.0 , ratio =  31.28  %
[DataGeneration] unique number of test data =  1.0 , ratio =  68.72  %
training_data.shape =  (532, 9)
validation_data.shape =  (227, 9)


In [ ]:
training_xdata = training_data[ :  , 0:-1]
training_tdata = training_data[ :  , [-1]]

validation_xdata = validation_data[ :  , 0:-1]
validation_tdata = validation_data[ :  , [-1]]

print("training_xdata = ", training_xdata.shape, ", training_tdata = ", training_tdata.shape)
print("validation_xdata = ", validation_xdata.shape, ", validation_tdata = ", validation_tdata.shape)

training_xdata =  (532, 8) , training_tdata =  (532, 1)
validation_xdata =  (227, 8) , validation_tdata =  (227, 1)


In [ ]:
input_nodes = training_data.shape[1]
hidden_nodes = 10
output_nodes = 1

In [ ]:
X = tf.placeholder(tf.float32, [None, 8])  # 8개 입력노드
T = tf.placeholder(tf.float32, [None, 1])  # 1개 정답노드

W2 = tf.Variable(tf.random_normal([input_nodex, hidden_nodes]))  # 가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_nodes]))     # 바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))  # 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))     # 바이어스 노드

In [ ]:
z2 = tf.matmul(X, W2) + b2
a2 = tf.sigmoid(z2)

z3 = tf.matmul(a2, W3) + b3
a3 = tf.sigmoid(z3)

loss = -tf.reduce_mean( T*tf.log(a3) + (1-T)*tf.log(1-a3))

In [ ]:
learning_rate = 1e-1
epochs = 50

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [ ]:
predicted = tf.cast(a3 > 0.5, dtype=tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype=tf.float32))

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(epochs):
    for step in range(len(training_data)):
      loss_val, _ = sess.run([loss, train], feed_dict={X: training_xdata, T: training_tdata})

      if (int)(0.05*step) == 0:
        print("step = ", step, ", loss_val = ", loss_val)

  y_val, predicted_val, accuracy_val = sess.run([a3, predicted, accuracy], feed_dict={X: validation_xdata, T: validation_tdata})

  print("\ny_val.shape = ", y_val.shape, ", predicted_val = ", predicted_val.shape)
  print("\nAccuracy = ", accuracy_val)

step =  0 , loss_val =  0.88598967
step =  1 , loss_val =  0.8757799
step =  2 , loss_val =  0.8660111
step =  3 , loss_val =  0.8566677
step =  4 , loss_val =  0.8477347
step =  5 , loss_val =  0.8391968
step =  6 , loss_val =  0.83103925
step =  7 , loss_val =  0.82324725
step =  8 , loss_val =  0.8158065
step =  9 , loss_val =  0.8087029
step =  10 , loss_val =  0.8019228
step =  11 , loss_val =  0.7954527
step =  12 , loss_val =  0.7892796
step =  13 , loss_val =  0.7833909
step =  14 , loss_val =  0.7777742
step =  15 , loss_val =  0.77241766
step =  16 , loss_val =  0.7673099
step =  17 , loss_val =  0.76243967
step =  18 , loss_val =  0.7577963
step =  19 , loss_val =  0.7533695
step =  0 , loss_val =  0.63725966
step =  1 , loss_val =  0.6372273
step =  2 , loss_val =  0.63719505
step =  3 , loss_val =  0.6371627
step =  4 , loss_val =  0.6371304
step =  5 , loss_val =  0.6370981
step =  6 , loss_val =  0.63706577
step =  7 , loss_val =  0.6370334
step =  8 , loss_val =  0.6370